# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import json
import random
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxend module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = False
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController



Approxend module not found, cannot use gamepad


In [2]:
experiment = "demonstration_collector"
run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_robot_controller = Config().get_experiment("controllers", exp["robot_controller"])
exp_camera_controller = Config().get_experiment("controllers", exp["camera_controller"])
exp_keyboard_controller = Config().get_experiment("controllers", exp["keyboard_controller"])




***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration_collector\collect_freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration_collector/collect_freeform successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/f

In [3]:
# find the demonstration path
# _, task_dir = ui_choose_task(offer_task_creation=True)

task_dir = exp.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

# the demonstration dictionary
#description = {}
#description["name"] = exp["name"]
#description["task"] = exp["name"]
#description["trim-from"] = 1 # counter number from where we need to take the data, default from beginning
#description["trim-to"] = -1 # counter number to where we need to take the demonstration, default -1 means to the end
#description["success"] = False # was the demonstration successful
#description["quality"] = 0.0 # numerical quality metric of the demonstration, 0.0 total failure, 1.0 perfect success
# description["text-annotation"] = "" # text annotation

# Write the overall file
#file_overall = pathlib.Path(demo_dir, "_demonstration.json")
#with open(file_overall, "w") as f:
    #str_description = json.dumps(description)
    #f.write(str_description)
#    json.dump(description, f)


Ok, demonstration with go into
	 c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\demonstration_collector\collect_freeform\2025_05_29__16_15_02


In [ ]:

# start the demonstration
    
# print("====== Starting the demonstration ========")

# the robot position controller
# robot_controller = PositionController(Config()["robot"]["usb_port"]) 
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None

#img_size = Config()["robot"]["saved_image_size"]
# (256, 256) # was (128, 128)
# camera_controller = CameraController(devices = Config()["robot"]["active_camera_list"], img_size = img_size)


In [5]:

camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True
#cameratracker = None
# the XBox controller - we are using the control loop from this one
# controller = "program"


cap0 works


In [ ]:

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(
        camera_controller=camera_controller, controller=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir, task_name=task_dir.name)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(camera_controller=camera_controller, controller=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "program":
    program_controller = ProgramController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    
    # create wpcount 
    waypoints = []
    wpcount = 10
    while True:
        norm = [0] * 6
        for df in range(6):
            norm[df] = random.random()                
        rp = RobotPosition.from_normalized_vector(norm)
        if RobotPosition.limit(rp):
            print(f"added waypoint {rp}")
            waypoints.append(rp)
        if len(waypoints) >= wpcount:
            break

    program_controller.waypoints = waypoints
    program_controller.interactive_confirm = False


    demo_recorder = DemonstrationRecorder(camera_controller=camera_controller, controller=program_controller, robot_controller=
                                robot_controller, save_dir=demo_dir, task_name=task_dir.name)

    program_controller.demonstration_recorder = demo_recorder
    program_controller.control()


NameError: name 'robot_controller' is not defined